In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Global variables
SNAPSHOT_DATE = '2042-06-22'

def non_zero_count(series):
    return (series > 0).sum()

# Data preprocessing

In [ ]:
# Importing the dataset
merchants = pd.read_excel('data/raw/merchants.xlsx')
payments = pd.read_excel('data/raw/payments.xlsx')
print(merchants.columns)
print(payments.columns)

In [ ]:
# '0.0' is the mssing value in the dataset
merchants_processed = merchants.replace(0, np.nan)
print(merchants_processed.shape)
print(merchants_processed.isnull().sum())

# drop the rows with nan values
merchants_processed = merchants_processed.dropna()
print(merchants_processed.shape)
merchant_ids = [str(id) for id in merchants_processed['merchant'].values]
print(len(merchant_ids))


In [ ]:
# payments dataset, only keep the rows with merchant id in merchant_ids
print(payments.shape)
payments_processed = payments.copy()
payments_processed['merchant'] = payments_processed['merchant'].astype(str)
payments_processed = payments[payments['merchant'].isin(merchant_ids)]
print(payments_processed.shape)
print(payments_processed['merchant'].nunique())

# payments_processed['date'] = pd.to_datetime(payments_processed['date'])
# payment date range
print(payments_processed['date'].min())
print(payments_processed['date'].max())

# Convert 'date' to datetime and 'month' to a period
payments_processed['date'] = pd.to_datetime(payments_processed['date'])
payments_processed['month'] = payments_processed['date'].dt.to_period('M')

## Define Outcome
Binary. By the last observation date, denote 1 if the merchant has used Subscriptions; 0 otherwise.

In [ ]:
outcome = payments_processed.groupby('merchant').agg(sub_min = ('subscription_volume', min))
outcome['label'] = (outcome['sub_min'] > 0).astype(int)
outcome = outcome.drop(columns=['sub_min']).reset_index()


## Aggregate Statistics

In [ ]:
# Aggregate payments by month for each merchant with sum, mean, max, and custom non-zero count
monthly_subscription_stats = payments_processed.groupby(['merchant', 'month'])['subscription_volume'].\
    agg(
        total_volume='sum',  # Total volume over the month
        average_volume='mean',  # Average volume per day
        peak_volume='max',  # Maximum volume in a single day
        active_days=non_zero_count  # Number of days with non-zero subscription
    )
monthly_checkout_stats = payments_processed.groupby(['merchant', 'month'])['checkout_volume'].\
    agg(
        total_volume='sum',  # Total volume over the month
        average_volume='mean',  # Average volume per day
        peak_volume='max',  # Maximum volume in a single day
        active_days=non_zero_count  # Number of days with non-zero subscription
    )
monthly_payment_link_stats = payments_processed.groupby(['merchant', 'month'])['payment_link_volume'].\
    agg(
        total_volume='sum',  # Total volume over the month
        average_volume='mean',  # Average volume per day
        peak_volume='max',  # Maximum volume in a single day
        active_days=non_zero_count  # Number of days with non-zero subscription
    )
monthly_total_stats = payments_processed.groupby(['merchant', 'month'])['total_volume'].\
    agg(
        total_volume='sum',  # Total volume over the month
        average_volume='mean',  # Average volume per day
        peak_volume='max',  # Maximum volume in a single day
        active_days=non_zero_count  # Number of days with non-zero subscription
    )

# Merge the three stats into one dataframe
monthly_stats= pd.merge(monthly_subscription_stats, monthly_checkout_stats, left_index=True, right_index=True, suffixes=('_sub', '_check'))
monthly_stats = pd.merge(monthly_stats, monthly_payment_link_stats, left_index=True, right_index=True, suffixes=('', '_link'))
monthly_stats = pd.merge(monthly_stats, monthly_total_stats, left_index=True, right_index=True, suffixes=('', '_total'))

monthly_stats.reset_index(inplace=True)
monthly_stats.columns = ['merchant', 'month', 
                         'sub_total', 'sub_avg', 'sub_peak', 'sub_active_days',
                         'check_total', 'check_avg', 'check_peak', 'check_active_days',
                         'link_total', 'link_avg', 'link_peak', 'link_active_days',
                         'total_total', 'total_avg', 'total_peak', 'total_active_days']

monthly_stats.head()

## Trends and Changes
Monthly growth rate

In [ ]:
monthly_stats.sort_values(by=['merchant', 'month'], inplace=True)
for product in ['sub', 'check', 'link', 'total']:
    monthly_stats[f'{product}_prev_total'] = monthly_stats.groupby('merchant')[f'{product}_total'].shift(1)
    # initialize the first month growth rate to 0
    monthly_stats[f'{product}_growth_rate'] = 0

    # case when previous month total is not 0
    mask = monthly_stats[f'{product}_prev_total'] != 0
    monthly_stats.loc[mask, f'{product}_growth_rate'] = monthly_stats.loc[mask, f'{product}_total'] / monthly_stats.loc[mask, f'{product}_prev_total'] - 1

    # case when previous month total is 0 and current month totol is 0
    mask = (monthly_stats[f'{product}_prev_total'] == 0) & (monthly_stats[f'{product}_total'] == 0)
    monthly_stats.loc[mask, f'{product}_growth_rate'] = 0

    # case when previous month total is 0 and current month totol is not 0
    mask = (monthly_stats[f'{product}_prev_total'] == 0) & (monthly_stats[f'{product}_total'] != 0)
    monthly_stats.loc[mask, f'{product}_growth_rate'] = 1

# Drop the columns for previous month's total volumes as they are no longer needed
monthly_stats.drop(columns=[f'{product}_prev_total' for product in ['sub', 'check', 'link', 'total']], inplace=True)

# Now monthly_stats includes the growth rate columns
print(monthly_stats[['merchant', 'month',
                     'sub_total', 'sub_growth_rate', 
                     'check_total', 'check_growth_rate', 
                     'link_total', 'link_growth_rate', 
                     'total_total', 'total_growth_rate']].head())

## Monthly variation

In [ ]:
volume_types = ['sub_total', 'check_total', 'link_total', 'total_total']
merchant_month_variation = pd.DataFrame(index = monthly_stats['merchant'].unique())
for volume_type in volume_types:
    # Standard Deviation
    std_col_name = f'{volume_type}_std'
    mean_col_name = f'{volume_type}_mean'
    cv_col_name = f'{volume_type}_cv'

    merchant_grouped = monthly_stats.groupby('merchant')[volume_type]

    merchant_month_variation[std_col_name] = merchant_grouped.std()
    merchant_month_variation[mean_col_name] = merchant_grouped.mean()
    merchant_month_variation[cv_col_name] = merchant_month_variation[std_col_name] / np.where(merchant_month_variation[mean_col_name] == 0, np.nan, merchant_month_variation[mean_col_name])

merchant_month_variation.reset_index(inplace=True)
# rename the index column to merchant
merchant_month_variation.rename(columns={'index': 'merchant'}, inplace=True)  
print(merchant_month_variation.head())

## Lasted 2 months engagement and Frequency
- Number of active days in the last 2 months
- Frequecny of transactions in the last 2 months

In [ ]:
payment_last_2month = payments_processed[payments_processed['month'] >= '2042-05']

In [ ]:
active_days = payment_last_2month[payment_last_2month['total_volume'] > 0].groupby('merchant')['date'].nunique()
payment_last_2month.sort_values(by=['merchant', 'date'], inplace=True)
payment_last_2month['day_between'] = payment_last_2month.groupby('merchant')['date'].diff().dt.days
avg_days_between = payment_last_2month.groupby('merchant')['day_between'].mean()

merchant_activity_stats = pd.DataFrame({
    'merchant': active_days.index,
    'active_days': active_days,
    'avg_days_between': avg_days_between
}).reset_index(drop=True)

## Merge data
merchants + aggregate statistics + trends and changes + monthly variation + lasted 2 months engagement and frequency

In [ ]:
print(merchants_processed.columns)
print(monthly_stats.columns)
print(merchant_month_variation.columns)
print(merchant_activity_stats.columns)

In [ ]:
# Pivot monthly_stats into a wide format
monthly_stats_wide = monthly_stats.pivot(index='merchant', columns='month').reset_index()
#print(monthly_stats_wide.head())
# Flatten the MultiIndex columns and join with an underscore, ensuring all elements are strings
monthly_stats_wide.columns = ['_'.join(map(str, col)).rstrip('_') for col in monthly_stats_wide.columns.values]
monthly_stats_wide.fillna(0, inplace=True)

# Now, monthly_stats_wide is in the wide format, with each merchant as a unique row and months as part of the column headers
#print(monthly_stats_wide.head())

# Ensure merchant column is the same type across all DataFrames for successful merge
monthly_stats_wide['merchant'] = monthly_stats_wide['merchant'].astype(str)
merchant_month_variation['merchant'] = merchant_month_variation['merchant'].astype(str)
merchant_activity_stats['merchant'] = merchant_activity_stats['merchant'].astype(str)

# Join the DataFrames on 'merchant'
wide_data = monthly_stats_wide.merge(merchant_month_variation, on='merchant', how='outer')
wide_data = wide_data.merge(merchant_activity_stats, on='merchant', how='outer')
print(wide_data.shape)

wide_data = wide_data.merge(merchants_processed, on = 'merchant', how = 'inner')
wide_data = wide_data.merge(outcome, on = 'merchant', how = 'inner')
# Now wide_data is a wide dataset where each row is a unique merchant
print(wide_data.shape)



In [ ]:
# find out the columns with missing values
missing_columns = wide_data.columns[wide_data.isnull().any()]
print(missing_columns)

In [ ]:
# save the full wide dataset
wide_data.to_csv('data/processed/wide_data.csv', index=False)
# save the data after removing the columns containing subscription information, sub_{}\
wide_data.drop(columns=[col for col in wide_data.columns if col.startswith('sub_')], inplace=True)
wide_data.to_csv('data/processed/wide_data_no_sub.csv', index=False)


In [ ]:
# print all column names
print(wide_data.columns)